<a href="https://bit.ly/Sports2D_Colab" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sports2D

When seeking to optimize a sports movement, it is often useful to compute joint positions, or joint and segment angles. \
**[Sports2D](https://github.com/davidpagnon/Sports2D)** offers a way to compute them from a video, on any platform, including your smartphone.\
Run cells one after another by clicking on the ▶️ button.

</br>

<img src="https://raw.githubusercontent.com/davidpagnon/Sports2D/main/Content/demo_gif.gif" title='Demonstration of Sports2D with OpenPose.'  width="760">


## Additional information

Please find more information about how it works under the hood [here](https://github.com/davidpagnon/Sports2D). 

While you're there, do not hesitate to hit the ⭐Star button, I would appreciate the support! \
If you use Sports2D, please cite:

     @misc{Pagnon2023,
       author = {Pagnon, David},
       title = {Sports2D - Angles from video},
       year = {2023},
       doi= {10.5281/zenodo.7903963},
       publisher = {GitHub},
       journal = {GitHub repository},
       howpublished = {\url{https://github.com/davidpagnon/Sports2D}},
     }

</br>

**Warning:**
- Results are only as good as the pose estimation algorithm, i.e., they are not perfect. Moreover, they will not be good if your video is blurry. Finally, they are acceptable only if the persons move in the 2D plane, from right to left or from left to right. If you need research-grade 3D markerless kinematics, consider using several cameras with [Pose2Sim](https://github.com/perfanalytics/pose2sim) instead.
- Your data will be sent to the Google servers, which do not follow the European GDPR requirements regarding privacy. If you don't want this, you can [run Sports 2D on your computer](https://github.com/davidpagnon/Sports2D) instead.
- The server disconnects after 90 minutes of idle time, and after a few hours in any case. Make sure you have downloaded your results beforehand.
- Unless you subscribed, there is a time limit on the usage of GPUs on Colab. However, these are only mandatory if you run OpenPose. If you run BlazePose, or if you want to analyze previously saved results, you can deactivate GPUs via `Runtime -> Change runtime type`


# Installation

In [1]:
#@markdown You can choose to: 
#@markdown - **Install just once**. Full install takes about 1.2 Go on your Google Drive (270 Mo for Python, 875 Mo for OpenPose),\
#@markdown but next times the environment should be set up in about 2 minutes, including on your smartphone.
#@markdown - **Install every time**. Full install takes about 22 minute (2 minutes for Python, 20 minutes for OpenPose), \
#@markdown but no connexion to Google Drive nor storage space is needed.
installation_type = 'Install once' #@param ["Install once", "Install every time"]

#@markdown <br>

#@markdown You can also choose to:
#@markdown - **Install OpenPose** if you need to go for accuracy and multi-person analysis.
#@markdown - **Not install OpenPose** for a much faster and lighter installation with BlazePose instead. 
install_openpose = 'Install OpenPose' #@param ["Install OpenPose", "Do not install OpenPose"]

import os, sys
from google.colab import files, drive

def connect_to_Gdrive():
  print("Connecting to Google Drive...")
  # Add GDrive notebooks to colab
  drive.mount('/content/drive')
  if not os.path.exists(sports2d_path):
    !mkdir -p '/content/drive/My Drive/Sports2D'
  else:
    print(f"{sports2d_path} already exists.")
    !cp -r $sports2d_path '/content/drive/My Drive'
    !rm -r $sports2d_path
  os.symlink('/content/drive/My Drive/Sports2D', sports2d_path)
  sys.path.insert(0, sports2d_path)

def install_Sports2D_in_GDrive():
  print("Installing Sports2D in Google Drive...")
  python_path = os.path.join(sports2d_path,'Sports2D_python')
  if not os.path.exists(python_path):
    # Install Python libraries in GDrive
    !PYTHONUSERBASE=$python_path pip install --user Sports2D
  else: 
    print('Sports2D already installed.')
  # Add the GDrive python installation path to PYTHONPATH
  sys.path.append(os.path.join(python_path,"lib/python3.10/site-packages"))

def install_openpose_func():
  print("Installing OpenPose in Google Drive...")
  git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
  if not os.path.exists(openpose_path):
    !mkdir -p $openpose_path
    # install new CMake because of CUDA10
    !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
    !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
    !git clone $git_repo_url
    !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
    # install system dependencies
    !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
    # build openpose # doing it twice seems to solve the issue
    !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`   
    !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`   
    # Install body_25b model # caffe model not available anymore -> revert to BODY_25 as default
    !wget posefs1.perception.cs.cmu.edu/OpenPose/models/pose/1_25BBkg/body_25b/pose_iter_XXXXXX.caffemodel -P /content/openpose/models/pose/body_25b
    !wget https://raw.githubusercontent.com/CMU-Perceptual-Computing-Lab/openpose_train/master/experimental_models/1_25BBkg/body_25b/pose_deploy.prototxt -P /content/openpose/models/pose/body_25b 
    # Get compatible CUDA and CuDNN versions in order to avoid `CudaSuccess (2 vs. 0) out of memory` error
    !apt -qq install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2
    # Move openpose folder to GDrive
    !mv /content/openpose/* $openpose_path && rm -r /content/openpose
    !rm -rf /content/openpose
    !chmod 755 $openpose_path/build/examples/openpose/openpose.bin
    # Copy libraries in system path
    !cp $openpose_path/build/src/openpose/libopenpose.so.1.7.0 /usr/local/lib
    !cp $openpose_path/build/caffe/lib/libcaffe.so.1.0.0 /usr/local/lib
    !sudo ldconfig
  else:
    print("OpenPose already installed.")
    print("Setting up the environment...")
    # Allow execution of OpenPose
    !chmod 755 $openpose_path/build/examples/openpose/openpose.bin
    # Install system dependencies
    !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev &> /dev/null
    # Get compatible CUDA and CuDNN versions in order to avoid `CudaSuccess (2 vs. 0) out of memory` error
    !apt -qq install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2 &> /dev/null
    # Copy libraries in system path
    !cp $openpose_path/build/src/openpose/libopenpose.so.1.7.0 /usr/local/lib
    !cp $openpose_path/build/caffe/lib/libcaffe.so.1.0.0 /usr/local/lib
    !sudo ldconfig

sports2d_path = '/content/Sports2D'
python_path = os.path.join(sports2d_path,'Sports2D_python')
openpose_path = os.path.join(sports2d_path,'Sports2D_openpose')
video_path = os.path.join(sports2d_path,'Sports2D_videos')
result_path = os.path.join(sports2d_path,'Sports2D_results')

# Use the right Python version
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
# GDrive tools
!apt-get -qq install xattr &> /dev/null

if installation_type == "Install every time":
  print('Installing Sports2D')
  !pip install sports2D
  if install_openpose == 'Install OpenPose':
    install_openpose_func()

elif installation_type == "Install once":
  connect_to_Gdrive()
  install_Sports2D_in_GDrive()
  if install_openpose == 'Install OpenPose':
    install_openpose_func()

print("Done.")

Connecting to Google Drive...
Mounted at /content/drive
Installing Sports2D in Google Drive...
Sports2D already installed.
Installing OpenPose in Google Drive...
OpenPose already installed.
Setting up the environment...
Done.


# Upload videos

In [47]:
#@markdown Upload your videos if needed, and convert them if needed.\
#@markdown **Note:** As this takes time, you should probably try to trim your videos beforehand.



def upload_videos():
  if not os.path.exists(video_path):
    !mkdir -p $video_path
  %cd $video_path
  uploaded = files.upload()

def convert_vertical_iphone_vids(vids):
  # Pose algorithms do not see the rotation in certain videos taken in portrait mode. Convert them
  for i, vid in enumerate(vids):
    iphone_portrait = !ffprobe -loglevel error -select_streams v:0 -show_entries stream_tags=rotate -of default=nw=1:nk=1 -i $vid
    if iphone_portrait:
      vid_out = os.path.splitext(vid)[0]+'_converted'+os.path.splitext(vid)[1]
      print(f'{vid} is rotated. Converting it to {vid_out}...')
      !ffmpeg -i $vid $vid_out -loglevel quiet
      vids[i] = vid_out
      os.remove(vid)

# upload videos
upload_videos()

# convert them if needed
vids = os.listdir(video_path)
vids = [f for f in vids if not f.startswith('.') and f!='logs.txt']
convert_vertical_iphone_vids(vids)

# list them
vids = os.listdir(video_path)
vids = [f for f in vids if not f.startswith('.')]
if vids==[]:
  if installation_type == 'Install every time':
    !cp /usr/local/lib/python3.10/dist-packages/Sports2D/Demo/demo.mp4 $video_path/demo.mp4
  if installation_type == 'Install once':
    !cp $python_path/lib/python3.10/site-packages/Sports2D/Demo/demo.mp4 $video_path/demo.mp4
  vids = ['demo.mp4']
print(f'{vids} uploaded and ready to be processed.')

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
/content/drive/My Drive/Sports2D/Sports2D_videos


['demo.mp4'] uploaded and ready to be processed.


# Run Sports2D


## Specify video to analyze


In [48]:
import toml

#@markdown Check left pane for a list of them.

# Retrieve Sports2D Config file
if not os.path.exists(result_path):
  !mkdir -p $result_path

if installation_type == 'Install every time':
  !cp /usr/local/lib/python3.10/dist-packages/Sports2D/Demo/Config_demo.toml $result_path/Config.toml
if installation_type == 'Install once':
  !cp $python_path/lib/python3.10/site-packages/Sports2D/Demo/Config_demo.toml $result_path/Config.toml
%cd $result_path
config_dict = toml.load('Config.toml')

video_name = 'demo.mp4' #@param {type:"string"}
video_name = os.path.basename(video_name)
pose_model = 'BODY_25'
config_dict.get('project').update({'video_dir':video_path})
config_dict.get('project').update({'video_file':video_name})
config_dict.get('project').update({'result_dir':result_path})
config_dict.get('pose').update({'pose_algo':'OPENPOSE'})
config_dict.get('pose').get('OPENPOSE').update({'openpose_model':pose_model})
config_dict.get('pose').get('OPENPOSE').update({'openpose_path':openpose_path})

video_full_path = os.path.join(video_path, video_name)
if os.path.isfile(video_full_path)==True: 
  print(f'Video found at {video_full_path}')
else: 
  print(f'Video not found at {video_full_path}')

/content/drive/My Drive/Sports2D/Sports2D_results
Video found at /content/Sports2D/Sports2D_videos/demo.mp4


## Settings (optional)

### Pose settings

In [39]:
#@markdown **OpenPose** supports multi-person analysis and is more accurate, but it requires a heavier install than **BlazePose**:
pose_algo = 'OpenPose' #@param ["OpenPose", "BlazePose"]

#@markdown <br>

#@markdown - **If OpenPose** algorithm is chosen.\ 
#@markdown   **BODY_25** is the standard model, but **BODY_25B** is more accurate:
if pose_algo == 'OpenPose':
  if install_openpose != 'Install OpenPose':
    print('WARNING: OpenPose not installed!')
  else:
    pose_model = 'BODY_25' #@param ["BODY_25B", "BODY_25"]
    config_dict.get('pose').update({'pose_algo':'OPENPOSE'})
    config_dict.get('pose').get('OPENPOSE').update({'openpose_model':pose_model})
    
#@markdown <br>

#@markdown - **If BlazePose** is chosen.
if pose_algo == 'BlazePose':
  model_complexity = 'Most accurate' #@param ["Most accurate", "Default", "Fastest"]
  model_complexity_dict = {'Most accurate':2, 'Default': 1, 'Fastest': 0}
  model_complexity_nb = model_complexity_dict[model_complexity]
  pose_model = 'BLAZEPOSE'
  
  config_dict.get('pose').update({'pose_algo':'BLAZEPOSE'})
  config_dict.get('pose').get('BLAZEPOSE').update({'model_complexity':model_complexity_nb})

### Angle settings

In [ ]:
#@markdown - Select **joint angles** among:
right_ankle = True #@param {type:"boolean"}
left_ankle = True #@param {type:"boolean"}
right_knee = True #@param {type:"boolean"}
left_knee = True #@param {type:"boolean"}
right_hip = True #@param {type:"boolean"}
left_hip = True #@param {type:"boolean"}
right_shoulder = True #@param {type:"boolean"}
left_shoulder = True #@param {type:"boolean"}
right_elbow = True #@param {type:"boolean"}
left_elbow = True #@param {type:"boolean"}

joint_angles_var = [right_ankle, left_ankle, right_knee, left_knee, right_hip, left_hip, right_shoulder, left_shoulder, right_elbow, left_elbow]
joint_angles_dict = {'Right ankle':right_ankle, 'Left ankle': left_ankle, 'Right knee':right_knee, 'Left knee':left_knee, 'Right hip':right_hip, 'Left hip':left_hip, 'Right shoulder':right_shoulder, 'Left shoulder':left_shoulder, 'Right elbow':right_elbow, 'Left elbow':left_elbow}
joint_angles_val = [key for key in joint_angles_dict.keys() if joint_angles_dict[key]]

config_dict.get('compute_angles').update({'joint_angles':joint_angles_val})

#@markdown <br>

#@markdown - Select **segment angles** among:
right_foot = True #@param {type:"boolean"}
left_foot = True #@param {type:"boolean"}
right_shank = True #@param {type:"boolean"}
left_shank = True #@param {type:"boolean"}
right_thigh = True #@param {type:"boolean"}
left_thigh = True #@param {type:"boolean"}
trunk = True #@param {type:"boolean"}
right_arm = True #@param {type:"boolean"}
left_arm = True #@param {type:"boolean"}
right_forearm = True #@param {type:"boolean"}
left_forearm = True #@param {type:"boolean"}

segment_angles_var = [right_foot, left_foot, right_shank, left_shank, right_thigh, left_thigh, trunk, right_arm, left_arm, right_forearm, left_forearm]
segment_angles_dict = {'Right foot':right_foot, 'Left foot': left_foot, 'Right shank':right_shank, 'Left shank':left_shank, 'Right thigh':right_thigh, 'Left thigh':left_thigh, 'Trunk':trunk, 'Right arm':right_arm, 'Left arm':left_arm, 'Right forearm':right_forearm, 'Left forearm':left_forearm}
segment_angles_val = [key for key in segment_angles_dict.keys() if segment_angles_dict[key]]

config_dict.get('compute_angles').update({'segment_angles':segment_angles_val})

### Advanced pose settings

In [ ]:
#@markdown Do not run pose estimation again if files already exist:
overwrite_pose = False #@param {type:"boolean"}
config_dict.get('pose_advanced').update({'overwrite_pose':overwrite_pose})

#@markdown <br>

#@markdown Choose whether to save images and video of pose estimation:
save_img = True #@param {type:"boolean"}
save_vid = True #@param {type:"boolean"}
config_dict.get('pose_advanced').update({'save_img':save_img})
config_dict.get('pose_advanced').update({'save_vid':save_vid})

#@markdown <br>

#@markdown Interpolate gaps only if they are smaller than XX frames:
interp_gap_smaller_than = 5 #@param {type:"slider", min:0, max:100, step:1}

#@markdown <br>

#@markdown Choose whether to filter results:
filter = True #@param {type:"boolean"}
filter_type = 'butterworth' #@param ["butterworth", "gaussian", "LOESS", "median"]
config_dict.get('pose_advanced').update({'filter':filter})
config_dict.get('pose_advanced').update({'filter_type':filter_type})

#@markdown <br>

#@markdown - If Butterworth filter is chosen, specify its order and cut-off frequency (in Hz):
order = 4 #@param {type:"integer"}
config_dict.get('pose_advanced').get('butterworth').update({'order':order})
cut_off_frequency = 4 #@param {type:"integer"}
config_dict.get('pose_advanced').get('butterworth').update({'cut_off_frequency':cut_off_frequency})

#@markdown <br>

#@markdown - If Gaussian filter is chosen, specify its sigma kernel (in pixel):
sigma_kernel = 1 #@param {type:"integer"}
config_dict.get('pose_advanced').get('gaussian').update({'sigma_kernel':sigma_kernel})

#@markdown <br>

#@markdown - If LOESS filter is chosen, specify the number of values used:
nb_values_used = 5 #@param {type:"integer"}
config_dict.get('pose_advanced').get('loess').update({'nb_values_used':nb_values_used})

#@markdown <br>

#@markdown - If Median filter is chosen, specify its kernel size (in frames):
kernel_size = 3 #@param {type:"integer"}
config_dict.get('pose_advanced').get('median').update({'kernel_size':kernel_size})

### Advanced angle settings

In [ ]:
#@markdown Choose whether to save images and video of angle estimation:
show_angles_on_img = True #@param {type:"boolean"}
show_angles_on_vid = True #@param {type:"boolean"}
config_dict.get('compute_angles_advanced').update({'show_angles_on_img':show_angles_on_img})
config_dict.get('compute_angles_advanced').update({'show_angles_on_vid':show_angles_on_img})

#@markdown <br>

#@markdown Choose whether to filter results:
filter = False #@param {type:"boolean"}
filter_type = 'butterworth' #@param ["butterworth", "gaussian", "LOESS", "median"]
config_dict.get('compute_angles_advanced').update({'filter':filter})
config_dict.get('compute_angles_advanced').update({'filter_type':filter_type})

#@markdown <br>

#@markdown - If Butterworth filter is chosen, specify its order and cut-off frequency (in Hz):
order = 4 #@param {type:"integer"}
config_dict.get('compute_angles_advanced').get('butterworth').update({'order':order})
cut_off_frequency = 4 #@param {type:"integer"}
config_dict.get('compute_angles_advanced').get('butterworth').update({'cut_off_frequency':cut_off_frequency})

#@markdown <br>

#@markdown - If Gaussian filter is chosen, specify its sigma kernel (in pixel):
sigma_kernel = 1 #@param {type:"integer"}
config_dict.get('compute_angles_advanced').get('gaussian').update({'sigma_kernel':sigma_kernel})

#@markdown <br>

#@markdown - If LOESS filter is chosen, specify the number of values used:
nb_values_used = 5 #@param {type:"integer"}
config_dict.get('compute_angles_advanced').get('loess').update({'nb_values_used':nb_values_used})

#@markdown <br>

#@markdown - If Median filter is chosen, specify its kernel size (in frames):
kernel_size = 3 #@param {type:"integer"}
config_dict.get('compute_angles_advanced').get('median').update({'kernel_size':kernel_size})

## Run analysis

In [49]:
#@markdown Let's go!

import logging, logging.handlers
logging.basicConfig(format='%(message)s', level=logging.INFO, force=True,
                    handlers = [logging.handlers.TimedRotatingFileHandler(os.path.join(result_path, 'logs.txt'), when='D', interval=7), logging.StreamHandler()]) 

from Sports2D import Sports2D
%cd $result_path
with open('Config.toml', "w") as config_file: toml.dump(config_dict, config_file)
Sports2D.detect_pose('Config.toml')
Sports2D.compute_angles('Config.toml')



---------------------------------------------------------------------
Detect pose for video demo.mp4
---------------------------------------------------------------------
Detecting 2D joint positions with OpenPose model BODY_25B, for demo.mp4.


/content/drive/My Drive/Sports2D/Sports2D_results


Sorting people across frames.
3 persons found.
Person 0: Interpolating missing sequences if they are smaller than 5 frames.
Person 0: Filtering with Butterworth filter, 4th order, 6 Hz..
Person 0: Saving csv position file in /content/drive/My Drive/Sports2D/Sports2D_results/demo_BODY_25B_person0_points.csv.
Person 1: Interpolating missing sequences if they are smaller than 5 frames.
Person 1: Filtering with Butterworth filter, 4th order, 6 Hz..
Person 1: Saving csv position file in /content/drive/My Drive/Sports2D/Sports2D_results/demo_BODY_25B_person1_points.csv.
Person 2: Interpolating missing sequences if they are smaller than 5 frames.
Person 2: Filtering with Butterworth filter, 4th order, 6 Hz..
Person 2: Saving csv position file in /content/drive/My Drive/Sports2D/Sports2D_results/demo_BODY_25B_person2_points.csv.
Saving images and video in /content/drive/My Drive/Sports2D/Sports2D_results.
Done.
Pose detection took 21.35 s.


----------------------------------------------------

# Retrieve results

## Show video

In [50]:
#@markdown Display last uploaded video after analysis:
show_last = 'Yes' #@param ["Yes", "No"]
#@markdown If "No", please type in the video you wish to display (check left pane for a list of them):
show_video_name = 'demo_BODY_25.mp4' #@param {type:"string"}

def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

if show_last == 'Yes':
  vid_to_show = os.path.join(result_path, os.path.splitext(video_name)[0] + '_' + pose_model + os.path.splitext(video_name)[1])
  if not os.path.isfile(vid_to_show):
    vid_to_show = os.path.join(result_path, os.path.splitext(video_name)[0] + '_' + pose_model + '.mp4')
else:
  vid_to_show = os.path.join(result_path, show_video_name)
  if not os.path.isfile(vid_to_show):
    vid_to_show = os.path.join(video_path, show_video_name)
  if not os.path.isfile(vid_to_show):
    vid_to_show = os.path.join('/content', show_video_name)
  if not os.path.isfile(vid_to_show):
    vid_to_show = show_video_name
print(vid_to_show)

# Check that the codec can be read in Colab
codec = !ffprobe -v error -select_streams v:0 -show_entries stream=codec_name -of default=noprint_wrappers=1:nokey=1 $vid_to_show
if codec[0] == 'mpeg4':
  print('Codec not supported: converting video...')
  vid_converted = os.path.join('/content', os.path.basename(vid_to_show))
  !ffmpeg -i $vid_to_show $vid_converted -y -loglevel quiet
  vid_to_show = vid_converted
show_local_mp4_video(vid_to_show, width=640, height=480)

/content/Sports2D/Sports2D_results/demo_BODY_25B.mp4
Codec not supported: converting video...


## Plot angles

In [51]:
import pandas as pd
import plotly.express as px

#@markdown Display knee angle results of the first person detected in the last uploaded video:
plot_last = 'No' #@param ["Yes", "No"]

#@markdown </br>

#@markdown If "No", please type in the name of the csv file you wish to plot, either of positions or angles (check left pane for a list of them):
plot_csv_name = 'demo_BODY_25_person1_angles.csv' #@param {type:"string"}
#@markdown And type in the names of the variables you are interested in:
plot_variable_names = "['Right knee','Left knee']" #@param {type:"string"}

#@markdown </br>

#@markdown Display table too:
display_table = 'Yes' #@param ["Yes", "No"]


if plot_last == 'Yes':
  csv_to_show = os.path.join(result_path, os.path.splitext(video_name)[0] + '_' + pose_model + '_angles.csv')
  if not os.path.isfile(csv_to_show):
    csv_to_show = os.path.join(result_path, os.path.splitext(video_name)[0] + '_' + pose_model + '_person0_angles.csv')
  if not os.path.isfile(csv_to_show):
    csv_to_show = os.path.join(result_path, os.path.splitext(video_name)[0] + '_converted' + pose_model + '_angles.csv')
  if not os.path.isfile(csv_to_show):
    csv_to_show = os.path.join(result_path, os.path.splitext(video_name)[0] + '_converted' + pose_model + '_person0_angles.csv')
else:
  csv_to_show = os.path.join(result_path, plot_csv_name)
  if not os.path.isfile(csv_to_show):
    csv_to_show = plot_csv_name

# Retrieve csv results
table = pd.read_csv(csv_to_show, index_col=0, header=[0,1,2,3])
table = table.droplevel([0,1], axis=1)
try:
  plot_variable_names = eval(plot_variable_names)
  plot_variable_names = ['Time']+plot_variable_names
  table_select = table[plot_variable_names]
  table_select.columns = [' '.join(col).strip() for col in table_select.columns.values]
  table_select = table_select.set_index(list(table_select)[0])
  fig = px.line(data_frame=table_select, width=1310, height=699)
  fig.show()
except:
  print('Variables could not be found in csv file')

if display_table=='Yes':
  display(table)

angs,Time,Right ankle,Left ankle,Right knee,Left knee,Right hip,Left hip,Right shoulder,Left shoulder,Right elbow,...,Left foot,Right shank,Left shank,Right thigh,Left thigh,Trunk,Right arm,Left arm,Right forearm,Left forearm
coords,seconds,dorsiflexion,dorsiflexion,flexion,flexion,flexion,flexion,flexion,flexion,flexion,...,horizontal,horizontal,horizontal,horizontal,horizontal,horizontal,horizontal,horizontal,horizontal,horizontal
0,0.000000,-0.241123,-32.858269,46.572561,117.195551,51.273033,58.250132,72.408274,-65.193720,97.543246,...,-113.188209,-138.647889,-170.329939,-92.075328,-53.134389,111.384521,-70.940087,-176.578241,26.603159,-156.711584
1,0.033272,-1.640537,-5.448134,42.540809,115.862802,44.732574,62.147512,73.266802,-68.388907,103.614683,...,-78.936309,-139.737898,-163.488175,-97.197089,-47.625373,109.772886,-68.662861,-178.161793,34.951821,-151.599967
2,0.066546,5.917359,-6.214535,38.573001,112.844432,37.911706,64.744014,72.840980,-69.789858,109.191693,...,-72.061129,-141.672392,-155.846594,-103.099391,-43.002162,107.746177,-68.170117,-177.536035,41.021576,-146.524487
3,0.099816,16.859324,-3.307130,34.998746,108.489234,31.744772,65.446093,71.097520,-69.092781,113.915877,...,-61.727504,-143.813539,-148.420374,-108.814792,-39.931140,105.377233,-69.462045,-174.470013,44.453832,-141.057924
4,0.133086,16.974726,3.070688,31.038999,103.681343,26.453887,65.421110,68.776424,-66.705854,117.566554,...,-48.513270,-144.787796,-141.583958,-113.748796,-37.902615,103.323725,-71.426260,-170.029579,46.140294,-135.800630
5,0.166358,-6.726901,6.501624,26.697781,98.222290,21.911286,65.131263,66.135585,-63.233036,119.267905,...,-38.465725,-144.454181,-134.967349,-117.756400,-36.745060,101.876323,-73.532101,-165.109359,45.735803,-130.436179
6,0.199630,-34.320082,7.469547,23.832420,91.419091,18.504542,63.364093,62.814381,-58.841166,116.477259,...,-31.225395,-144.139818,-128.694942,-120.307398,-37.275851,100.639943,-75.997559,-159.481110,40.479700,-123.109608
7,0.232901,-39.070025,8.290176,24.399648,83.718627,16.535110,60.473243,58.573395,-52.946636,98.509213,...,-24.970430,-145.307574,-123.260606,-120.907926,-39.541978,100.015221,-78.869641,-152.961858,19.639572,-112.814233
8,0.266173,-34.222035,8.354544,28.552062,76.887832,15.896061,59.472872,53.598154,-44.957761,41.643697,...,-20.392838,-148.365911,-118.747382,-119.813849,-41.859550,101.332422,-82.111756,-146.290183,-40.468059,-100.654271


## View in Google drive

In [52]:
#@markdown If you chose "Install once", results are already stored on your Google Drive.\
#@markdown If you chose "Install every time", results will first be copied there.

#@markdown `Warning:` If this leads you to a 404 error, run this cell again after a few seconds.

# Buttons to link to Gdrive
import ipywidgets as widgets
from IPython.display import display, Javascript, clear_output, HTML
from google.colab import drive

# Create button
def Gdrive_results_onbuttonclicked(url):
  with output:
    display(Javascript(f'window.open("{url.tooltip}");'))

if installation_type == 'Install every time':
  connect_to_Gdrive()

os.chdir(result_path)
gdrive_result_id = !xattr -p 'user.drive.id' $result_path
gdrive_result_path = 'https://drive.google.com/drive/folders/' + gdrive_result_id[0]
button = widgets.Button(description="View in Google Drive", tooltip=gdrive_result_path, button_style='info')
button.on_click(Gdrive_results_onbuttonclicked)
output = widgets.Output()
display(button, output)

Button(button_style='info', description='View in Google Drive', style=ButtonStyle(), tooltip='https://drive.go…

Output()

## Download results

In [ ]:
#@markdown Download all results. \
#@markdown Alternatively, you can download them individually after finding them in the left pane.

# Compress result folder
!zip -r /content/Sports2D_results.zip /content/Sports2D/Sports2D_results

# Download zip file
from google.colab import files
files.download('/content/Sports2D_results.zip')